# Azure: attach image to an instance for inspection

https://gitlab.com/cloudigrade/cloudigrade/issues/484


## Copying the Image

Every VM in Azure is stored as a blob, unfortunately these blobs can't be copy over directly to cloudigrade's storage account. The first step needed is to create a blob snapshot of the blob that we would like to copy. 

The CLI command for this is: 

```
az storage blob snapshot 
--container-name vhds \
--name rhel7220181004163509.vhd \
--account-key \
--account-name cloudigradegroupdisks
```

This Blob snapshot can then be copied over to the cloudigrade storage account.

```
az storage blob copy start \
--destination-blob copied-blob.vhd \
--destination-container container2 \
--account-name storagestuff2  \
--account-key \
--source-account-key \
--source-account-name  cloudigradegroupdisks \
--source-container vhds \
--source-blob rhel7220181004163509.vhd \
--source-snapshot 2018-10-05T13:17:15.4164963Z  
```

We can see the blob copy status. The UI will lie and not show copy failures :/

```
az storage blob show \
-container-name container2 \
--account-name=storagestuff2 \
--account-key= \
--name copied-blob.vhd
```
Note that when the blob snapshot is copied over to the cloudigrade account, it's treated as a blob, not a blob snapshot. 

~~This blob can be used to create an image:~~

~~Creating the image needs us to provide the `os-type`, but this information can be found when we describe the running instances:~~


In [8]:
from datetime import datetime, timedelta
from getpass import getpass
from pprint import pprint
from urllib.parse import quote, urlparse

from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.compute.models import Image, ImageOSDisk, ImageStorageProfile, OperatingSystemStateTypes
from azure.storage.blob.baseblobservice import BaseBlobService
from azure.storage.blob.sharedaccesssignature import BlobSharedAccessSignature
from azure.storage.blob.models import BlobPermissions
from azure.storage.file.fileservice import FileService


In [13]:
SAFE_CHARS = '/()$=\',~'
# this is Bizhang's account
source_password = getpass(prompt="Password of source storage account")
source_account = 'cloudigradegroupdisks'
source_container_name = 'vhds'

source_client = BaseBlobService(account_name=source_account, 
                               account_key=source_password)


# this is Brad's account
# Note: To mount this storage account it must exist under the AKS resource group
dest_password = getpass(prompt="Password of destination storage account")
dest_account = 'storageaccount4'

# Copy the blob the a file share. We can mount
dest_client = FileService(account_name=dest_account, account_key=dest_password)



In [ ]:
blobs = source_client.list_blobs(container_name=source_container_name)

#Create a shared access signature for the source container
expiry = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%SZ')
sas = BlobSharedAccessSignature(account_name=source_account, 
                                account_key=source_password)

source_sas = sas.generate_container(
    source_container_name, 
    expiry=expiry, 
    protocol='https', 
    permission=BlobPermissions(read=True)
)


for blob in blobs:
    
    # if the blob is not a vhd, don't copy it.
    if not blob.name.endswith('vhd'):
        continue
    
    # Create a snapshot of the source blob
    snapshot = source_client.snapshot_blob(
        container_name=source_container_name,
        blob_name=quote(blob.name, SAFE_CHARS)
    )

    # Get the source blob snapshot url
    source_blob_url = source_client.make_blob_url(
        container_name=source_container_name, 
        blob_name=quote(blob.name, SAFE_CHARS), 
        sas_token=source_sas,
        snapshot=snapshot.snapshot
    )
    
    # Copy the blob snapshot to a fileshare. 
    copy_blob_operation = dest_client.copy_file(
        file_name=blob.name, 
        copy_source=source_blob_url,
        share_name='fileshare',
        directory_name='test'
    )  

Azure has a couple of different storage solutions we might use to mount and inspect the VHD we copied over from the client. But these storage solutions work differently with each azure tool (for example we can't straight up mount the blob storage with in an Azure Container Instance)

Azure's storage solutions are:

1. Blob
2. File (this is basically more structured blob storage)
3. Disk (this is a disk created from the VHD)

Here is an guide to the difference between these: https://docs.microsoft.com/en-us/azure/storage/common/storage-decide-blobs-files-disks
If we use Blob or File, we get a bunch of .vhd files, that we have to mount in order to inspect. 

As a note, Azure says: `[Use blob storage when] you want to be able to access application data from anywhere.` This is a lie. Most of the Azure Tools we look at does not support accessing blobs.


## Azure-Container-Instance

https://docs.microsoft.com/en-us/azure/container-instances/
https://docs.microsoft.com/en-us/cli/azure/container

This is Azure's lightweight container tool. The only storage option it supports is mounting an Azure Fileshare (and git, and secret, but we can't really use those).

We can create a container with the fileshare mounted using

```
az container create \
    --resource-group MC_eastus-group_cloudigrade-cluster_eastus \
    --name test1 \
    --image werwty/docker-nginx-libguestfs \
    --azure-file-volume-account-name cloudigradeaksstorage \
    --azure-file-volume-account-key \
    --azure-file-volume-share-name fileshare \
    --azure-file-volume-mount-path /mnt/cloudigrade/
```

We can then see the .vhd files on the container at`/mnt/cloudigrade/files/`. 
But, these .vhd files cannot be mounted in the container without more permission than we can get from ACI. Specifically we need the `cap_sys_admin` capability in oreder to mount things in a container. Unfortunately ACI does not give us a way to connect to the underlying Docker API, or run priviledged container [0]


## Azure-Kubernetes-Service

https://docs.microsoft.com/en-us/azure/aks/

Spinning up one of these gives us a kubernetes cluster. AKS allows us to mount disks and fileshares, but not blobs.

There are some problems with mounting disks: 
1. Azure only allows 4 volumes to be mounted to a cluster at once. We would have a batch size of 4.
2. Azure mounts the disks at /dev/sd{c-f} but we are not told where it mounts a specific disk. We can assume they'll be mounted sequentually, but this is assuming Azure behavior remains sane. 
3. Creating a disk from a Blob requires a `disk-size` argument. If this is not provided Azure defaults to 1024GB per disk. 



An alternate solution is to mount a fileshare, and then mount each vhd in that fileshare individually for inspection.
As a POC, I've done this with a privileged pod, we can narrow down the pod capabilities in the future. 


1. Create AKS group in Azure
2. Get the credentials locally, so we can authenticate with kubectl

    ` az aks get-credentials --resource-group eastus-group --name houndigrade-cluster`
    
    To view the kubernetes cluster run: `az aks browse --resource-group eastus-group --name houndigrade-cluster`

3. Create a kubernetes secret so kubernetes has access to our storage account:
`kubectl create secret generic azure-secret --from-literal=azurestorageaccountname=storageaccount4 --from-literal=azurestorageaccountkey=`
4. Create a pod using this yaml:
```
apiVersion: v1
kind: Pod
metadata:
  name: cloudipod
spec:
  containers:
  - image: fedora:latest
    name: cloudipod
    command: ["/bin/bash", "-ec", "while :; do echo '.'; sleep 5 ; done"]
    securityContext:
      privileged: true
      allowPrivilegeEscalation: true
    resources:
      requests:
        cpu: 200m
        memory: 256Mi
      limits:
        cpu: 250m
        memory: 256Mi
    volumeMounts:
      - name: azure
        mountPath: /mnt/azure
  volumes:
  - name: azure
    azureFile:
      secretName: azure-secret
      shareName: fileshare
      readOnly: false
```
5. In the pod itself we can run the following commands to see the internal vhd disk structure:

```
[root@cloudipod3 /]# cd /mnt/azure/test/
[root@cloudipod3 test]# ls
bleh  rhel7220181004163509.vhd
[root@cloudipod3 test]# kpartx -av rhel7220181004163509.vhd 
add map loop2p1 (253:4): 0 1024000 linear 7:2 2048
add map loop2p2 (253:5): 0 66082816 linear 7:2 1026048
[root@cloudipod3 test]# mkdir /mnt/cloudigrade
[root@cloudipod3 test]# mount /dev/mapper/loop2p2 /mnt/cloudigrade/
[root@cloudipod3 test]# ls -la /mnt/cloudigrade/
total 28
dr-xr-xr-x. 17 root root 4096 Sep  7  2017 .
drwxr-xr-x   1 root root 4096 Oct 19 03:42 ..
lrwxrwxrwx.  1 root root    7 Sep  7  2017 bin -> usr/bin
drwxr-xr-x.  2 root root    6 Sep  7  2017 boot
drwxr-xr-x.  2 root root    6 Sep  7  2017 dev
drwxr-xr-x. 86 root root 8192 Oct  4 20:42 etc
drwxr-xr-x.  3 root root   20 Oct  4 20:38 home
lrwxrwxrwx.  1 root root    7 Sep  7  2017 lib -> usr/lib
lrwxrwxrwx.  1 root root    9 Sep  7  2017 lib64 -> usr/lib64
drwxr-xr-x.  2 root root    6 May 25  2015 media
drwxr-xr-x.  4 root root   33 Oct  4 20:37 mnt
drwxr-xr-x.  3 root root   15 Sep  7  2017 opt
drwxr-xr-x.  2 root root    6 Sep  7  2017 proc
dr-xr-x---.  3 root root   97 Sep  7  2017 root
drwxr-xr-x.  2 root root    6 Sep  7  2017 run
lrwxrwxrwx.  1 root root    8 Sep  7  2017 sbin -> usr/sbin
drwxr-xr-x.  2 root root    6 May 25  2015 srv
drwxr-xr-x.  2 root root    6 Sep  7  2017 sys
drwxrwxrwt.  7 root root   88 Oct 18 03:39 tmp
drwxr-xr-x. 13 root root 4096 Sep  7  2017 usr
drwxr-xr-x. 20 root root 4096 Oct  4 20:37 var
[root@cloudipod3 test]# 
```


[0] https://docs.microsoft.com/en-us/azure/container-instances/container-instances-troubleshooting#cannot-connect-to-underlying-docker-api-or-run-privileged-containers